In [1]:
from xcube_geodb.core.geodb import GeoDBClient
from dotenv import load_dotenv
import os

load_dotenv()

g = GeoDBClient(
  server_url=os.environ['SERVER_URL'],
  server_port=os.environ['SERVER_PORT'],
  client_id=os.environ['CLIENT_ID'],
  client_secret=os.environ['CLIENT_SECRET'],
  auth_aud=os.environ['AUTH_DOMAIN'],
)

g.whoami

'geodb_juanpedro_ep'

In [60]:
ds = g.get_my_collections('stac_test')
ds

,owner,database,collection
0,geodb_b34bfae7-9265-4a3e-b921-06549d3c6035,stac_test,ge_train_tier_1_labels
1,geodb_b34bfae7-9265-4a3e-b921-06549d3c6035,stac_test,ge_train_tier_1_source
2,geodb_b34bfae7-9265-4a3e-b921-06549d3c6035,stac_test,ge_train_tier_2_labels
3,geodb_b34bfae7-9265-4a3e-b921-06549d3c6035,stac_test,ge_train_tier_2_source
4,geodb_b34bfae7-9265-4a3e-b921-06549d3c6035,stac_test,ties_ai_challenge_test


In [61]:
gdf = g.get_collection('ge_train_tier_1_labels', database='stac_test')
gdf.head()

,id,created_at,modified_at,geometry,label:description,area,label:type,label:properties,label:overviews,license,...,label:counts_building:material_foundation,label:counts_building:material_foundation;cement_block,label:counts_amenity_metal,label:counts_amenity_water_point,label:counts_building:material_bamboo,label:counts_amenity_fire_station,label:counts_amenity_internet_cafe,label:counts_amenity_microfinance_bank,label:counts_amenity_mobil_money_agent,label:counts_amenity_money_transfer
0,1,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((-0.24297 5.63902, -0.23897 5.64169, ...",Geojson building labels for scene 665946,acc,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,2,-1,-1,-1,-1,-1,3
1,2,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((-0.24987 5.61797, -0.24832 5.61838, ...",Geojson building labels for scene a42435,acc,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,3,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((-0.22609 5.60782, -0.22707 5.60957, ...",Geojson building labels for scene ca041a,acc,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,4,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((-0.20863 5.58212, -0.20386 5.57954, ...",Geojson building labels for scene d41d81,acc,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,5,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((39.27611 -6.81095, 39.27609 -6.81095...",Geojson building labels for scene 0a4c40,dar,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,6,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((39.21153 -6.80836, 39.21101 -6.80781...",Geojson building labels for scene 353093,dar,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,11,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((32.62648 0.25642, 32.62713 0.25620, ...",Geojson building labels for scene 4e7c7f,kam,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,7,2023-05-22T23:11:04.474479+00:00,None,"MULTIPOLYGON (((39.24911 -6.83028, 39.24884 -6...",Geojson building labels for scene 42f235,dar,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,12,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((-10.79025 6.32765, -10.79076 6.32746...",Geojson building labels for scene 207cc7,mon,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,13,2023-05-22T23:11:04.474479+00:00,None,"POLYGON ((-10.79241 6.33761, -10.79133 6.33837...",Geojson building labels for scene 401175,mon,vector,"['building', 'building:material', 'roof:materi...","[{'property_key': 'building', 'counts': [{'nam...",ODbL-1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [71]:
gdf = g.get_properties('ge_train_tier_1_labels', 'stac_test')
l = gdf[gdf['column_name'] == 'label:properties']
l

,database,collection,column_name,data_type
6,stac_test,ge_train_tier_1_labels,label:properties,character varying


In [72]:
gdf

,database,collection,column_name,data_type
0,stac_test,ge_train_tier_1_labels,id,integer
1,stac_test,ge_train_tier_1_labels,created_at,timestamp with time zone
2,stac_test,ge_train_tier_1_labels,modified_at,timestamp with time zone
3,stac_test,ge_train_tier_1_labels,label:description,character varying
4,stac_test,ge_train_tier_1_labels,area,character varying
...,...,...,...,...
206,stac_test,ge_train_tier_1_labels,label:counts_amenity_internet_cafe,integer
207,stac_test,ge_train_tier_1_labels,label:counts_amenity_microfinance_bank,integer
208,stac_test,ge_train_tier_1_labels,label:counts_amenity_mobil_money_agent,integer
209,stac_test,ge_train_tier_1_labels,label:counts_amenity_money_transfer,integer


In [55]:
g.get_my_databases()

,id,name,owner,iss
0,1038,geodb_juanpedro_ep,geodb_juanpedro_ep,


## STAC to GeoDB

In [15]:
import geopandas as gpd

gdf = gpd.read_file('sample.json')
gdf = gdf.rename(columns={'id': 'stac_id'})
gdf

,type,stac_version,stac_id,properties,links,assets,stac_extensions,geometry
0,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_a...,"{'area': 'acc', 'datetime': '2018-08-05T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"POLYGON ((-0.24297 5.63902, -0.23897 5.64169, ..."
1,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_a...,"{'area': 'acc', 'datetime': '2018-10-06T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"POLYGON ((-0.24987 5.61797, -0.24832 5.61838, ..."
2,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_a...,"{'area': 'acc', 'datetime': '2018-11-12T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"POLYGON ((-0.22609 5.60782, -0.22708 5.60957, ..."
3,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_a...,"{'area': 'acc', 'datetime': '2019-07-07T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"POLYGON ((-0.20863 5.58212, -0.20386 5.57954, ..."
4,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_d...,"{'area': 'dar', 'datetime': '2017-11-01T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"POLYGON ((39.27611 -6.81095, 39.27609 -6.81095..."
5,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_d...,"{'area': 'dar', 'datetime': '2015-04-20T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"POLYGON ((39.21153 -6.80836, 39.21101 -6.80781..."
6,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_d...,"{'area': 'dar', 'datetime': '2017-11-01T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"MULTIPOLYGON (((39.24911 -6.83028, 39.24884 -6..."
7,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_d...,"{'area': 'dar', 'datetime': '2015-04-17T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"POLYGON ((39.26926 -6.77841, 39.26918 -6.77806..."
8,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_d...,"{'area': 'dar', 'datetime': '2015-05-20T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"POLYGON ((39.23204 -6.79095, 39.23214 -6.78910..."
9,Feature,1.0.0,open_cities_ai_challenge_train_tier_1_labels_d...,"{'area': 'dar', 'datetime': '2017-11-01T00:00:...","[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{'labels': {'href': './labels.geojson', 'type'...",[https://stac-extensions.github.io/label/v1.0....,"MULTIPOLYGON (((39.27316 -6.80403, 39.27317 -6..."


In [16]:
g.drop_collection('eurosat_stac_test')

In [17]:
collection = {
                "type": "character varying",
                "stac_version": "character varying",
                "properties": "character varying",
                "stac_id": "character varying",
                "links": "character varying",
                "assets": "character varying",
                "stac_extensions": "character varying"
            }


g.create_collection('eurosat_stac_test', collection)

In [18]:
g.get_my_collections('eurosat_stac_test')

,collection


In [19]:
g.insert_into_collection('eurosat_stac_test', gdf)

Processing rows from 0 to 31


In [21]:
g.get_collection('eurosat_stac_test')

,id,created_at,modified_at,geometry,type,stac_version,properties,stac_id,links,assets,stac_extensions
0,1,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((-0.24297 5.63902, -0.23897 5.64169, ...",Feature,1.0.0,"{""area"": ""acc"", ""datetime"": ""2018-08-05T00:00:...",open_cities_ai_challenge_train_tier_1_labels_a...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
1,2,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((-0.24987 5.61797, -0.24832 5.61838, ...",Feature,1.0.0,"{""area"": ""acc"", ""datetime"": ""2018-10-06T00:00:...",open_cities_ai_challenge_train_tier_1_labels_a...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
2,3,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((-0.22609 5.60782, -0.22708 5.60957, ...",Feature,1.0.0,"{""area"": ""acc"", ""datetime"": ""2018-11-12T00:00:...",open_cities_ai_challenge_train_tier_1_labels_a...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
3,4,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((-0.20863 5.58212, -0.20386 5.57954, ...",Feature,1.0.0,"{""area"": ""acc"", ""datetime"": ""2019-07-07T00:00:...",open_cities_ai_challenge_train_tier_1_labels_a...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
4,5,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((39.27611 -6.81095, 39.27609 -6.81095...",Feature,1.0.0,"{""area"": ""dar"", ""datetime"": ""2017-11-01T00:00:...",open_cities_ai_challenge_train_tier_1_labels_d...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
5,6,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((39.21153 -6.80836, 39.21101 -6.80781...",Feature,1.0.0,"{""area"": ""dar"", ""datetime"": ""2015-04-20T00:00:...",open_cities_ai_challenge_train_tier_1_labels_d...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
6,7,2023-05-29T14:30:21.469068+00:00,None,"MULTIPOLYGON (((39.24911 -6.83028, 39.24884 -6...",Feature,1.0.0,"{""area"": ""dar"", ""datetime"": ""2017-11-01T00:00:...",open_cities_ai_challenge_train_tier_1_labels_d...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
7,8,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((39.26926 -6.77841, 39.26918 -6.77806...",Feature,1.0.0,"{""area"": ""dar"", ""datetime"": ""2015-04-17T00:00:...",open_cities_ai_challenge_train_tier_1_labels_d...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
8,9,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((39.23204 -6.79095, 39.23214 -6.78910...",Feature,1.0.0,"{""area"": ""dar"", ""datetime"": ""2015-05-20T00:00:...",open_cities_ai_challenge_train_tier_1_labels_d...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
9,12,2023-05-29T14:30:21.469068+00:00,None,"POLYGON ((-10.79025 6.32765, -10.79076 6.32746...",Feature,1.0.0,"{""area"": ""mon"", ""datetime"": ""2018-10-05T00:00:...",open_cities_ai_challenge_train_tier_1_labels_m...,"[ { ""rel"": ""collection"", ""href"": ""..\/collecti...","{""labels"": {""href"": ""./labels.geojson"", ""type""...","[""https://stac-extensions.github.io/label/v1.0..."
